In [44]:
import re

def oldest_migration_matrix(num_populations):
    oldest_matrix = f"//Migration matrix {num_populations - 1}"
    temp = [["0"] * num_populations for _ in range(num_populations)]
    oldest_matrix = [oldest_matrix] + [" ".join(row) for row in temp]
    return oldest_matrix
def population_name(index = None, split_SF=False, ghost_present=False):
    if not split_SF:
        populations = ["SF", "WRM"]
    else:
        populations = ["SFWC", "SFP", "WRM"]
    if ghost_present:
        populations.append("G")
    if index == None:
        index = list(range(len(populations)))
        return [populations[i] for i in index]
    return populations[index]
def current_migration_matrix(num_populations, **kwargs):
    current_matrix = ["//Migration matrix 0"]
    
    for i in range(1, num_populations + 1):
        matrix_row = []
        for j in range(1, num_populations + 1):
            if i == j:
                matrix_at_i_j = "0"
            else:
                from_pop = population_name(index = j-1, **kwargs)
                to_pop = population_name(index = i-1, **kwargs)
                matrix_at_i_j = f"MIG_{from_pop}to{to_pop}"
                
            matrix_row.append(matrix_at_i_j)
        current_matrix.append(" ".join(matrix_row))
    return(current_matrix)
def matrix_generation(num_pops, divergence_events, **kwargs):
    subsequent_matrix = current_migration_matrix(num_populations=num_pops, **kwargs)
    output = [subsequent_matrix] 
    
    # # loop throught all divergence events to create matrices going backward in time
    for i in reversed(range(len(divergence_events))):
        event = divergence_events[i]
        subsequent_matrix_index = re.sub(r'T.* ([0-9])$', r'\1', event)
        if int(subsequent_matrix_index) == num_pops - 1:
            subsequent_matrix = oldest_migration_matrix(num_pops)
        else:
            coalescing_population = re.sub(r"^TDIV_[a-zA-Z]*to([a-zA-Z]*) [0-9].*$", r"\1", event)
           # do this to a matrix without the title element
            trimmed_matrix = subsequent_matrix
            title = trimmed_matrix.pop(0)
            pattern = r"MIG_{}to[A-Z]*|MIG_[A-Z]*to{}".format(coalescing_population, coalescing_population)
            subsequent_matrix = [
                re.sub(
                    pattern, 
                    "0", 
                    line
                )
                for line in trimmed_matrix
            ]
            subsequent_matrix.insert(0, title)

        # output = output + subsequent_matrix
        output.append(subsequent_matrix)
    return output

num_pops = 3
divergence_events = ["TDIV_WRMtoSFWC 0 2 1 RES_WRMtoSFWC 0 2", "TDIV_SFWCtoSFP 1 0 1 RES_SFWCtoSFP 0 1"]
split_SF = True
add_ghost = False

matrix = matrix_generation(num_pops, divergence_events, split_SF=split_SF, ghost_present=add_ghost)
print(matrix)
# for el in matrix:
#     print(el)
[
    ['0 MIG_SFPtoSFWC MIG_WRMtoSFWC', 
     'MIG_SFWCtoSFP 0 MIG_WRMtoSFP', 
     'MIG_SFWCtoWRM MIG_SFPtoWRM 0'
     ], 
     ['//Migration matrix 0', 
      '0 0 MIG_WRMtoSFWC', 
      '0 0 0', 
      'MIG_SFWCtoWRM 0 0'
      ], 
      [
          '//Migration matrix 2', 
          '0 0 0', 
          '0 0 0', 
          '0 0 0'
          ]]

[["0 MIG_['SFP']to['SFWC'] MIG_['WRM']to['SFWC']", "MIG_['SFWC']to['SFP'] 0 MIG_['WRM']to['SFP']", "MIG_['SFWC']to['WRM'] MIG_['SFP']to['WRM'] 0"], ['//Migration matrix 0', "0 MIG_['SFP']to['SFWC'] MIG_['WRM']to['SFWC']", "MIG_['SFWC']to['SFP'] 0 MIG_['WRM']to['SFP']", "MIG_['SFWC']to['WRM'] MIG_['SFP']to['WRM'] 0"], ['//Migration matrix 2', '0 0 0', '0 0 0', '0 0 0']]


[['0 MIG_SFPtoSFWC MIG_WRMtoSFWC',
  'MIG_SFWCtoSFP 0 MIG_WRMtoSFP',
  'MIG_SFWCtoWRM MIG_SFPtoWRM 0'],
 ['//Migration matrix 0', '0 0 MIG_WRMtoSFWC', '0 0 0', 'MIG_SFWCtoWRM 0 0'],
 ['//Migration matrix 2', '0 0 0', '0 0 0', '0 0 0']]